### 1. 创建`provider`

In [1]:
from web3 import Web3
# 利用Infura的rpc节点连接以太坊网络
# 准备 infura API 可以参考https://github.com/AmazingAng/WTF-Solidity/tree/main/Topics/Tools/TOOL02_Infura/readme.md
INFURA_GEORLI_URL = 'https://goerli.infura.io/v3/5cd64b21ee2e4a1fa149862681338bbf' 
Provider = Web3(Web3.HTTPProvider(INFURA_GEORLI_URL))
Provider.is_connected()

True

### 2. 定义合约地址与合约`abi`

In [2]:
# 测试网络WETH地址
addressWETH = Provider.to_checksum_address('0xb4fbf271143f4fbf7b91a5ded31805e42b2208d6')
# WETH合约abi
abiWETH = '[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"guy","type":"address"},{"name":"wad","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"src","type":"address"},{"name":"dst","type":"address"},{"name":"wad","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"wad","type":"uint256"}],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"dst","type":"address"},{"name":"wad","type":"uint256"}],"name":"transfer","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"},{"name":"","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":true,"name":"guy","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":true,"name":"dst","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"dst","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Deposit","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Withdrawal","type":"event"}]'

### 3. 声明`WETH`合约实例

In [3]:
WETHcontract = Provider.eth.contract(address=addressWETH,abi=abiWETH)

### 4. 获取过去10个区块内的`Transfer`事件，并打印出第一个

In [4]:
# 得到当前block
block = Provider.eth.block_number
print(f"当前区块高度：{block}")
print("打印事件详情：")
logs = WETHcontract.events.Transfer.get_logs(fromBlock=block-10, toBlock=block)
print(logs[0])

当前区块高度：10184882
打印事件详情：
AttributeDict({'args': AttributeDict({'src': '0xf94d0c47D2Cb6f18622426DB4F454D58CcbDFD1E', 'dst': '0x270D8F77cf968954F0F616ACB9301fCACD06bD02', 'wad': 344770451507748340}), 'event': 'Transfer', 'logIndex': 50, 'transactionIndex': 8, 'transactionHash': HexBytes('0xb510924469fc8d011a4b9c79730e1bfce7eb7a2b72f27bdfb288f21d1d30f029'), 'address': '0xB4FBF271143F4FBf7B91A5ded31805e42b2208d6', 'blockHash': HexBytes('0x9162f004869e236b6fd6c3a94cfc8b3f3099f4ffcc7d8e9bfb6648b2538034c8'), 'blockNumber': 10184872})


### 5. 读取事件的解析结果

In [5]:
# 解析Transfer事件的数据（变量在args中）
print("解析事件：")
print(f"地址{logs[0].args.src} 转账 {Provider.from_wei(logs[0].args.wad, 'ether')} WETH 到地址{logs[0].args.dst}")

解析事件：
地址0xf94d0c47D2Cb6f18622426DB4F454D58CcbDFD1E 转账 0.34477045150774834 WETH 到地址0x270D8F77cf968954F0F616ACB9301fCACD06bD02


### 6. 利用argument_filters参数过滤事件

In [6]:
# 读取所有src地址为指定地址的Transfer事件，输出解析后的事件
filteredLogs = WETHcontract.events.Transfer.get_logs(fromBlock=block-10, toBlock=block,argument_filters={'src':logs[0].args.src})
print("打印从指定地址出发的Transfer事件详情：")
for l in filteredLogs:
    print(f"地址{l.args.src} 转账 {Provider.from_wei(l.args.wad, 'ether')} WETH 到地址{l.args.dst}")

打印从指定地址出发的Transfer事件详情：
地址0xf94d0c47D2Cb6f18622426DB4F454D58CcbDFD1E 转账 0.34477045150774834 WETH 到地址0x270D8F77cf968954F0F616ACB9301fCACD06bD02
地址0xf94d0c47D2Cb6f18622426DB4F454D58CcbDFD1E 转账 0.100994870419913553 WETH 到地址0x620BbfB5E551Bd84cAE59B6d26311888e148901D
